In [ ]:
!pip install opencv-python mediapipe

顔のパーツに対応したnodeの割り振りとそのidの可視化

In [ ]:
import cv2
import mediapipe as mp

# MediaPipe の初期化
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 顔のランドマーク描画用スタイル
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Webカメラを起動
# 0の時たまにスマホのカメラが起動するので１
cap = cv2.VideoCapture(1)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("カメラからフレームを取得できませんでした。")
            break

        # パフォーマンス向上のため画像を一時的に「書き込み不可」に
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        # 画像を「書き込み可能」に戻す
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # ランドマークの描画
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=drawing_spec)
            #顔のnodeの横にランドマークidを描画
            h,w,_=image.shape
            for idx,landmark in enumerate(face_landmarks.landmark):
                x=int(landmark.x*w)
                y=int(landmark.y*h)
                if 0 <= x < w and 0 <= y < h:
                    cv2.putText(image,str(idx),(x,y),cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.3, (0, 255, 0), 1, cv2.LINE_AA)

        # 表示
        cv2.imshow('MediaPipe FaceMesh', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

目のクマの要素だけ可視化

In [ ]:
import cv2
import mediapipe as mp

# MediaPipe の初期化
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 顔のランドマーク描画用スタイル
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Webカメラを起動
# 0の時たまにスマホのカメラが起動するので１
cap = cv2.VideoCapture(1)
##両目のクマの部分のみをプロットする際に繋ぐnodeを指定（一つ下の行のものは右目のもののみ)
under_eye_connection=[(362,382),(382,381),(381,380),(380,374),(374,373),(373,390),(390,249),(249,263),(263,359),(359,446),(446,261),(261,448),(448,449),(449,450),(450,451),(451,452),(452,341),(341,362)]

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("カメラからフレームを取得できませんでした。")
            break

        # パフォーマンス向上のため画像を一時的に「書き込み不可」に
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        # 画像を「書き込み可能」に戻す
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # ランドマークの描画
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=under_eye_connection,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=drawing_spec)
            #顔のnodeの横にランドマークidを描画
            # h,w,_=image.shape
            # for idx,landmark in enumerate(face_landmarks.landmark):
            #     x=int(landmark.x*w)
            #     y=int(landmark.y*h)
            #     if 0 <= x < w and 0 <= y < h:
            #         cv2.putText(image,str(idx),(x,y),cv2.FONT_HERSHEY_SIMPLEX, 
            #                         0.3, (0, 255, 0), 1, cv2.LINE_AA)

        # 表示
        cv2.imshow('MediaPipe FaceMesh', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()